In [7]:
# initialization
import matplotlib.pyplot as plt
import numpy as np
import time

# importing Qiskit
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, assemble

# import basic plot tools
from qiskit.visualization import plot_histogram

In [8]:
backendObject = Aer.get_backend('qasm_simulator')

In [9]:
def buildBernsteinVaziraniCircuit(numberOfQubits, bitMask) :
    
    s = bitMask
    n = numberOfQubits
    
    # We need a circuit with n qubits, plus one auxiliary qubit
    # Also need n classical bits to write the output to
    bv_circuit = QuantumCircuit(n+1, n)

    # put auxiliary in state |->
    bv_circuit.h(n)
    bv_circuit.z(n)

    # Apply Hadamard gates before querying the oracle
    for i in range(n):
        bv_circuit.h(i)
    
    # Apply barrier 
    bv_circuit.barrier()

    # Apply the inner-product oracle
    s = s[::-1] # reverse s to fit qiskit's qubit ordering
    for q in range(n):
        if s[q] == '0':
            bv_circuit.i(q)
        else:
            bv_circuit.cx(q, n)
        
    # Apply barrier 
    bv_circuit.barrier()

    #Apply Hadamard gates after querying the oracle
    for i in range(n):
        bv_circuit.h(i)

    # Measurement
    for i in range(n):
        bv_circuit.measure(i, i)
        
    return bv_circuit

In [10]:
def runBernsteinVaziraniAlgorithm(numberOfQubits, bitMask) :
    timeBefore = time.perf_counter()
    bernsteinVaziraniCircuit = buildBernsteinVaziraniCircuit(numberOfQubits, bitMask)
    quantumObject = assemble(bernsteinVaziraniCircuit)
    results = backendObject.run(quantumObject).result()
    timeAfter = time.perf_counter()
    totalElapsedTime = timeAfter - timeBefore   
    formattedTotalElapsedTime = "{:.4f}".format(totalElapsedTime)  
    print("[" + str(numberOfQubits) + "] Time: " + str(formattedTotalElapsedTime))


In [11]:
runBernsteinVaziraniAlgorithm(10, '1100101100')
 

[10] Time: 0.0141
